# Contexte
Le gouvernement péruvien vient d’autoriser les aventuriers en quête de trésors à explorer les 85 182
km² du département de la Madre de Dios. Vous devez réaliser un système permettant de suivre les
déplacements et les collectes de trésors effectuées par les aventuriers. Le gouvernement péruvien
étant très à cheval sur les bonnes pratiques de code, il est important de réaliser un code de qualité,
lisible, et maintenable (oui, ça veut dire avec des tests) !

In [1]:
import re
import numpy as np
#from numpy import zeros,array

In [62]:
nbAdventurers = 0
tabAdventurers = []    # np.empty((2,5), list)
print(tabAdventurers)
gameMap = None

[]


## Lire le fichier d'entrée

In [63]:
def readFile(inputPath, outputPath):
    inputFile = open(inputPath, 'r')
    outputFile = open(outputPath, 'w')
    outputStrAcommeA = "# {A comme Aventurier} - {Nom de l’aventurier} - {Axe horizontal} - {Axe vertical} - {Orientation} - {Nb. trésors ramassés} \n"
    global nbAdventurers
    global tabAdventurers
    nbCellsWidth = 0
    nbCellsHeight = 0
    global gameMap
    xMontain = None
    yMontain = None
    xTreasure = None
    yTreasure = None
    nbTreasures = None
    
    
    while True:
        # Get next line from file
        line = inputFile.readline()
        if not line:
            print("break while true")
            break
        
        #print(line)
        if re.search("^(A|a).*-", line) == None:
            if re.search("^(#).*{(A|a).*comme", line):
                outputFile.write(outputStrAcommeA)
            elif re.search("^(#).*{(T|t).*comme", line):
                commentT = "# {T comme Trésor} - {Axe horizontal} - {Axe vertical} - {Nb. de trésors restants}"
                outputFile.write(commentT)
            else:
                # Carte
                if re.search("^(C|c).*-.*\d", line):
                    nbCellsWidth = int(re.findall("\d+", line)[0])
                    nbCellsHeight = int(re.findall("\d+", line)[1])
                    gameMap = np.empty((nbCellsHeight,nbCellsWidth), list)
                    outputFile.write(line)
                # Montagne    
                if re.search("^(M|m).*-.*\d", line):
                    xMontain = int(re.findall("\d+", line)[0])
                    yMontain = int(re.findall("\d+", line)[1])
                    gameMap[yMontain][xMontain] = "M"
                    outputFile.write(line)
                # Trésor
                if re.search("^(T|t).*-.*\d", line):
                    xTreasure = int(re.findall("\d+", line)[0])
                    #print("x treasure : " + str(xTreasure))
                    yTreasure = int(re.findall("\d+", line)[1])
                    #print("y treasure : " + str(yTreasure))
                    nbTreasures = int(re.findall("\d+", line)[2])
                    #print("nb treasure : " + str(nbTreasures))
                    gameMap[yTreasure][xTreasure] = ["T", xTreasure, yTreasure, nbTreasures]
                    #outputFile.write(line)
                                          
                #print("write line.")
                #print(line)
                #outputFile.write(line)
            
            
        else:
            # Adventurer(s)
            nbAdventurers += 1
            listElems = []
            listElems.append("A")
            nameAdv = re.findall("[a-zA-Z]{2,}", line)[0]
            listElems.append(nameAdv)
            columnAdv = int(re.findall("\d+", line)[0])
            listElems.append(columnAdv)
            lineAdv = int(re.findall("\d+", line)[1])
            listElems.append(lineAdv)
            orientationAdv = re.findall("\s+(N|n|S|s|E|e|O|o){1}\s+", line)[0]
            listElems.append(orientationAdv)
            motionSequence = re.findall("[a-zA-Z]{2,}", line)[1]
            listElems.append(motionSequence)
            nbTreasuresFound = 0
            listElems.append(nbTreasuresFound)
            print(listElems)
            indice = nbAdventurers-1
            tabAdventurers.append(listElems)
            print(tabAdventurers[0])
            gameMap[lineAdv][columnAdv] = listElems
                        
            print("output character")
            txtAname = str(re.findall("^A|a.*-.*-", line)) #+ "{{coordonnées}} {{nb trésors trouvés}}"
            outputFile.write(txtAname)
            
            #print("# nb adventurers : " + str(nbAdventurers))
            #print("name : " + str(nameAdv))
            #print("column number : " + str(columnAdv))
            #print("line number : " + str(lineAdv))
            #print("orientation : "+str(orientationAdv))
            #print("motion sequence :"+str(motionSequence))
               
    #for item in tabAdventurers:
        #print("item :: " + str(item))            
    inputFile.close()
    outputFile.close()
    print("all files closed")
    
    print("gameMap")
    print(".....................")
    print(gameMap)

In [64]:
cheminEntree = './ressources/input01.map'
cheminSortie = './ressources/output01.map'

In [65]:
readFile(cheminEntree, cheminSortie)

break while true
all files closed
gameMap
.....................
[[None None None]
 [None 'M' None]
 [None None 'M']
 [list(['T', 0, 3, 2]) list(['T', 1, 3, 1]) None]]


In [66]:
print(tabAdventurers)
print("nb Adventurers :", nbAdventurers)
print(gameMap)

[]
nb Adventurers : 0
[[None None None]
 [None 'M' None]
 [None None 'M']
 [list(['T', 0, 3, 2]) list(['T', 1, 3, 1]) None]]


In [67]:
def isInsideMap(y=int,x=int):
    gameMapHeight = gameMap.shape[0]
    gameMapWidth = gameMap.shape[1]
    
    if y < gameMapHeight and y >= 0 and x < gameMapWidth and x >= 0:
        return True
    else:
        return False

In [68]:
def getNewOrientation(orientation,nextMovement):
    if nextMovement != 'A':
        
        if nextMovement == 'D':
            if orientation == 'N':
                orientation = 'E'
            elif orientation == 'S':
                orientation = 'O'
            elif orientation == 'E':
                orientation = 'S'
            elif orientation == 'O':
                orientation = 'N'
        elif nextMovement == 'G':
            if orientation == 'N':
                orientation = '0'
            elif orientation == 'S':
                orientation = 'E'
            elif orientation == 'E':
                orientation = 'N'
            elif orientation == 'O':
                orientation = 'S'
            
    return orientation

In [71]:
def doSimulation(nbAdventurers, tabAdventurers, gameMap):
    gmHeight = gameMap.shape[0]
    gmWidth = gameMap.shape[1]
    print("Simulation...")
    nbAdvNoMov = 0
    infLoop = True
    
    while infLoop:
        for i in range(len(tabAdventurers)):
            print(tabAdventurers[i])
            print("next move ")
            print(tabAdventurers[i][5])
            peopleName = tabAdventurers[i][1]
            coordX = int(tabAdventurers[i][2])
            coordY = int(tabAdventurers[i][3])
            orientation = tabAdventurers[i][4]
            movementsList = tabAdventurers[i][5]
            if movementsList:
                nextMovement = movementsList[0]
            else:
                #print("BREAKKKKKKKKKK !!!!!!!!!!!!!!!")
                #break
                nbAdvNoMov = nbAdvNoMov +1
                if nbAdvNoMov == nbAdventurers:
                    infLoop = False
                    break
                    
            print(peopleName)
            #print(orientation)
            #print(nextMovement)
            
            if len(movementsList) >= 1:
                
                # Fonction orientationChange(actualOrientation,movement)   'D' or 'G'
                newOrientation = getNewOrientation(orientation,nextMovement)
                if orientation != newOrientation:
                    tabAdventurers[i][4] = newOrientation
        
                if newOrientation == 'S':
                    newCoordY = coordY + 1
                    newCoordX = coordX
                    #coordY = coordY + 1
                elif newOrientation == 'E':
                    newCoordX = coordX + 1
                    newCoordY = coordY
                    #coordX = coordX + 1
                elif newOrientation == 'N':
                    newCoordY = coordY - 1
                    newCoordX = coordX
                    #coordY = coordY - 1
                elif newOrientation == 'O':
                    newCoordX = coordX - 1
                    newCoordY = coordY
                    #coordX = coordX - 1
                    
                # Cell is empty
                if isInsideMap(newCoordY,newCoordX) and gameMap[newCoordY][newCoordX] == None:
                    gameMap[coordY][coordX] = None
                    tabAdventurers[i][3] = newCoordY # coordY    # newCoordY
                    tabAdventurers[i][2] = newCoordX # add 23:01
                    # remove movement
                    tabAdventurers[i][5] = tabAdventurers[i][5][1:]
                    #gameMap[newCoordY][newCoordX] = tabAdventurers[i]
                # Treasure in cell
                elif isInsideMap(newCoordY,newCoordX) and gameMap[newCoordY][newCoordX][0] == 'T':
                    nbTreasures = int(gameMap[newCoordY][newCoordX][3])
                    if nbTreasures >= 1:
                        tabAdventurers[i][6] = tabAdventurers[i][6] + 1
                        gameMap[coordY][coordX][3] = int(gameMap[coordY][coordX][3]) - 1
                    # remove movement
                    tabAdventurers[i][5] = tabAdventurers[i][5][1:]
                else:
                    # Cell is a mountain
                    # remove movement
                    tabAdventurers[i][5] = tabAdventurers[i][5][1:]
                    
                # remove movement
                #tabAdventurers[i][5] = tabAdventurers[i][5][1:]
                #gameMap[newCoordY][newCoordX] = tabAdventurers[i]                    
                
            #for elm in range(len(tabAdventurers[i][5])):
            #tabAdventurers[i][5] = tabAdventurers[i][5][1:]   # [1:]
            #print(tabAdventurers[i][5])
        if nbAdvNoMov == nbAdventurers:
            infLoop = False
        #break
        
    # Write output file
    with open(cheminSortie, "a+") as file_output:
        #commentTreasure = "# {T comme Trésor} - {Axe horizontal} - {Axe vertical} - {Nb. de trésors restants}"
        #file_output.write("\n")
        #file_output.write(commentTreasure)
        file_output.write("\n")
        for i in range(gmHeight):
            for j in range(gmWidth):
                print("i,j : ", i,j)
                if gameMap[i][j]:
                    #print("gameMap[i][j] != None !!!")
                    #print(gameMap[i][j])
                    #print(gameMap[i][j][0])
                    if gameMap[i][j][0] == 'T':
                        strTreasure = str(gameMap[i][j]).strip('[]')
                        strTreasure = str(strTreasure.replace(',',' -'))
                        strTreasure = str(strTreasure.replace('\'',''))
                        file_output.write(strTreasure)
                        file_output.write("\n")
                    
        #file_output.write()
        
        if len(tabAdventurers) >= 1:
            commentA = "# {A comme Aventurier} - {Nom de l’aventurier} - {Axe horizontal} - {Axe vertical} - {Orientation} - {Nb. trésors ramassés}"
            file_output.write("\n")
            file_output.write(commentA)
            for i in range(gmHeight):
                for j in range(gmWidth):
                    if gameMap[i][j] != None:
                        if gameMap[i][j][0] == 'A':
                            strAdventurer = str(gameMap[i][j]).strip('[]')
                            strAdventurer = str(strAdventurer.replace(',','-'))
                            file_output.write(strAdventurer)
            
        file_output.close()
    print("RETURNNnnnnnnnnn !!!!!!!!!!!!!!! §§§§§§§")
    return

In [72]:
doSimulation(nbAdventurers, tabAdventurers, gameMap)

Simulation...
i,j :  0 0
i,j :  0 1
i,j :  0 2
i,j :  1 0
i,j :  1 1
i,j :  1 2
i,j :  2 0
i,j :  2 1
i,j :  2 2
i,j :  3 0
i,j :  3 1
i,j :  3 2
RETURNNnnnnnnnnn !!!!!!!!!!!!!!! §§§§§§§


In [57]:
gmHeight = gameMap.shape[0]
print(gmHeight)
gmWidth = gameMap.shape[1]
print(gmWidth)

#for i in range(gmHeight):
    #for j in range(gmWidth):
        #print(i, j)

4
3


In [40]:
tabAdventurers

[]

In [41]:
gameMap

array([[None, None, None],
       [None, 'M', None],
       [None, None, 'M'],
       [list(['T', 0, 3, 2]), list(['T', 1, 3, 1]), None]], dtype=object)

In [17]:
len(gameMap)

4